# Extracting roadside ditches using Whitebox Tools and Python

#### Conceptual Method
1) Applies a Difference-from-Mean Elevation (DFME) filter to an input DEM.
2) Thresholds the DFME raster to extract grid cells where DFME < -0.15.
3) Buffers a road vector layer to create a road mask.
4) Converts the buffered road vector to a raster matching the DEM resolution and extent.
5) Multiplies the road buffer raster by the thresholded DFME raster to isolate ditches near roads.

#### Inputs
1) User-defined elevation raster

#### Outputs
1) dfme.tif: Difference-from-Mean Elevation raster.
2) dfme_thresholded.tif: Raster showing areas below -0.15 DFME.
3) roads_buffered.shp: Buffered road vector.
4) road_mask.tif: Rasterized road buffer.
5) final_result.tif: Output raster highlighting ditches near roads.

Imports

In [ ]:
import os
import whitebox
import geopandas as gpd
import rasterio
from rasterio.mask import mask
from rasterio.features import rasterize
import numpy as np

Path

In [ ]:
# Initialize WhiteboxTools
wbt = whitebox.WhiteboxTools()
wbt.verbose = False  # Set to True for debugging

# Define file paths
dem_path = "input_dem.tif"
dfme_path = "dfme.tif"
thresholded_dfme_path = "dfme_thresholded.tif"
road_vector_path = "roads.shp"
buffered_road_vector_path = "roads_buffered.shp"
road_raster_path = "road_mask.tif"
final_output_path = "final_result.tif"

1) DFME Filtering

- Uses WhiteboxTools to compute the DFME on the DEM.
- The filter size (e.g., 11x11) can be adjusted based on the dataset.

In [ ]:
# Step 1: Apply Difference-from-Mean Elevation (DFME) Filter
wbt.diff_from_mean_elev(dem_path, dfme_path, filter_size=11)  # Adjust filter size if needed

2) Thresholding DFME

- Extracts all cells where DFME < -0.15 (indicating low areas, likely ditches).

In [ ]:
# Step 2: Threshold DFME Raster to Extract Low Cells
threshold_value = -0.15  # Adjust this based on DEM units
wbt.raster_calculator(f"{dfme_path} < {threshold_value}", output=thresholded_dfme_path)

3) Buffering Road Vector

- Uses GeoPandas to buffer the roads by 10 meters.
- Adjusts for different road widths if necessary.

In [ ]:
# Step 3: Buffer Road Vector
buffer_distance = 10  # Buffer distance in meters (adjust as needed)
roads = gpd.read_file(road_vector_path)
roads_buffered = roads.copy()
roads_buffered["geometry"] = roads.buffer(buffer_distance)
roads_buffered.to_file(buffered_road_vector_path)

4) Rasterizing Buffered Road Layer

- Converts the buffered road vector to a raster with the same resolution and extent as the DEM.
- Assigns 1 to roads and 0 elsewhere.

In [ ]:
# Step 4: Convert Buffered Road Vector to Raster
with rasterio.open(dem_path) as dem:
    meta = dem.meta.copy()
    transform = dem.transform
    width, height = dem.width, dem.height

    # Rasterize buffered road layer
    road_shapes = [(geom, 1) for geom in roads_buffered["geometry"]]
    road_raster = rasterize(
        road_shapes, out_shape=(height, width), transform=transform, fill=0, dtype="uint8"
    )

    # Save road mask raster
    with rasterio.open(road_raster_path, "w", **meta) as dst:
        dst.write(road_raster, 1)



5) Multiplying the Road Raster by the Thresholded DFME Raster

- Ensures only ditches within the buffered road area are included.

In [ ]:
# Step 5: Multiply Road Buffer Raster by Thresholded DFME Raster
with rasterio.open(thresholded_dfme_path) as dfme, rasterio.open(road_raster_path) as road:
    dfme_data = dfme.read(1)
    road_data = road.read(1)

    # Multiply both rasters
    final_result = dfme_data * road_data

    # Save the final output
    meta.update(dtype="float32")
    with rasterio.open(final_output_path, "w", **meta) as dst:
        dst.write(final_result, 1)

print("Processing complete. Final result saved as:", final_output_path)

## Dynamic thresholding approaches


| Method | Pros | Cons |
| :--- | :----: | :---: |
| *Standard Deviation Approach* | Adapts to variations in DFME values, simple to compute | May not work well for skewed distributions |
| *Percentile-Based* | Works well for varying elevation ranges | Assumes a fixed % of pixels are ditches |
| *Local Adaptive Thresholding* | Works well for large or complex terrains | Computationally expensive |
| *K-Means Clustering* | Fully data-driven, no need to specify a threshold | Requires good separation in values |

#### 1. Statistical Approach: Using Standard Deviations from the Mean
 - Instead of a fixed threshold (e.g., -0.15), you can analyze the DFME raster’s statistical distribution and set the threshold based on standard deviations.

*Why Use This?*
- It adapts to local terrain variations by defining "low areas" relative to the dataset.
- Works well when ditches are not a fixed depth across the study area.

In [ ]:
import rasterio
import numpy as np

# Load DFME Raster
dfme_path = "dfme.tif"

with rasterio.open(dfme_path) as src:
    dfme_data = src.read(1)  # Read first band
    dfme_data[dfme_data == src.nodata] = np.nan  # Ignore NoData values

# Compute mean and standard deviation of DFME
mean_dfme = np.nanmean(dfme_data)
std_dfme = np.nanstd(dfme_data)

# Define threshold dynamically (e.g., 1.5 standard deviations below mean)
dynamic_threshold = mean_dfme - 1.5 * std_dfme  # Adjust factor as needed

print(f"Dynamic Threshold for DFME: {dynamic_threshold}")

# Apply threshold
thresholded_dfme = (dfme_data < dynamic_threshold).astype("uint8")

# Save thresholded DFME raster
output_thresholded_dfme = "dfme_dynamic_thresholded.tif"
meta = src.meta.copy()
meta.update(dtype="uint8")

with rasterio.open(output_thresholded_dfme, "w", **meta) as dst:
    dst.write(thresholded_dfme, 1)

print("Saved dynamically thresholded DFME raster:", output_thresholded_dfme)


#### 2. Percentile-Based Thresholding
- Instead of relying on absolute values or standard deviations, another method is to threshold based on percentiles. This approach ensures that you extract a consistent proportion of the lowest DFME values.

*Why Use This?*
- Works well for datasets with variable z-units or different elevation ranges.
- Ensures a consistent fraction of the landscape is considered as ditches.

In [ ]:
percentile_cutoff = 5  # Extracts lowest 5% of values

# Compute the threshold using percentiles
dynamic_threshold = np.nanpercentile(dfme_data, percentile_cutoff)

print(f"Dynamic Threshold (5th percentile): {dynamic_threshold}")

# Apply threshold
thresholded_dfme = (dfme_data < dynamic_threshold).astype("uint8")

# Save raster as before
output_thresholded_dfme = "dfme_percentile_thresholded.tif"
with rasterio.open(output_thresholded_dfme, "w", **meta) as dst:
    dst.write(thresholded_dfme, 1)


#### 3. Local Adaptive Thresholding
- Instead of using a single threshold for the entire dataset, you can segment the DEM into smaller tiles or use a moving window to compute localized thresholds.

*Why Use This?*
- Adapts to local variations in elevation, making it ideal for large or heterogeneous study areas.
- More computationally intensive but useful when ditches have varying depths across the region.

In [ ]:
from skimage.filters import threshold_local

# Define block size (must be odd)
block_size = 101  # Adjust based on DEM resolution

# Compute adaptive threshold
adaptive_threshold = threshold_local(dfme_data, block_size, method='gaussian', offset=0)

# Apply threshold
thresholded_dfme = (dfme_data < adaptive_threshold).astype("uint8")

# Save raster as before
output_thresholded_dfme = "dfme_local_thresholded.tif"
with rasterio.open(output_thresholded_dfme, "w", **meta) as dst:
    dst.write(thresholded_dfme, 1)


#### 4. Using Clustering (K-Means) to Find Thresholds
- If the ditch pixels form a distinct cluster in the DFME histogram, you can use K-means clustering to separate low values from the rest.

*Why Use This?*
- Works well when ditch pixels naturally form a distinct class in the elevation difference histogram.
- No need to predefine a threshold; it finds the best cutoff automatically.

In [ ]:
from sklearn.cluster import KMeans

# Flatten and remove NaN values
valid_pixels = dfme_data[~np.isnan(dfme_data)].reshape(-1, 1)

# Perform K-Means with 3 clusters (adjust as needed)
kmeans = KMeans(n_clusters=3, random_state=42)
kmeans.fit(valid_pixels)

# Use the lowest cluster mean as threshold
dynamic_threshold = min(kmeans.cluster_centers_)[0]

print(f"Dynamic Threshold from Clustering: {dynamic_threshold}")

# Apply threshold
thresholded_dfme = (dfme_data < dynamic_threshold).astype("uint8")

# Save raster as before
output_thresholded_dfme = "dfme_kmeans_thresholded.tif"
with rasterio.open(output_thresholded_dfme, "w", **meta) as dst:
    dst.write(thresholded_dfme, 1)
